In [46]:
%load_ext autoreload
%autoreload 2

from tweedejaars_project import *
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.preprocessing import *
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.neural_network import *
import xgboost as xgb
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer, MissingIndicator
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.tree._criterion import ClassificationCriterion
from sklearn.tree._splitter import BestSplitter
import math
from collections import namedtuple


df = load_df()

def add_is_balanced(df):
    df["is_balanced"] = df["min_price_published"].isna() & df["max_price_published"].isna()
    return df

df = add_is_balanced(df)

df["residual_load"] = df["forecast_demand"] - df["forecast_solar"] - df["forecast_wind"]
df["dispatch_diff"] = df["upward_dispatch_published"] - df["downward_dispatch_published"]
df["igcc_diff"] = df["igcc_contribution_up_published"] - df["igcc_contribution_down_published"]

df.columns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Index(['datetime', 'import_capacity', 'upward_dispatch_published',
       'downward_dispatch_published', 'min_price_published',
       'mid_price_published', 'max_price_published', 'minute_in_ptu',
       'min_ptu_price_known', 'max_ptu_price_known',
       'igcc_contribution_down_published', 'igcc_contribution_up_published',
       'settlement_price_bestguess', 'time_since_last_two_sided',
       'two_sided_daily_count', 'vwap_avg', 'vwap_std', 'vwap_median',
       'vwap_qty_sum', 'vwap_max', 'hvq_delta', 'PTU', 'target_two_sided_ptu',
       'settlement_price_realized', 'naive_strategy_action', 'forecast_wind',
       'forecast_solar', 'forecast_demand', 'ptu_id',
       'target_two_sided_ptu_alt', 'target_two_sided_ptu_realtime',
       'target_two_sided_ptu_flip', 'fix_two_sided_ptu',
       'fix_two_sided_ptu_alt', 'fix_two_sided_ptu_realtime',
       'fix_two_sided_ptu_flip', 'is_balanced', 'residual_load',
       'dispatch_diff', 'igcc_diff'],
      dtype='object')

In [76]:
features = [
    "import_capacity",
    "min_price_published",
    "mid_price_published",
    "max_price_published",
    "min_ptu_price_known",
    "max_ptu_price_known",
    "upward_dispatch_published",
    "downward_dispatch_published",
    "time_since_last_two_sided",
    "two_sided_daily_count",
    "PTU",
    "naive_strategy_action",
    "vwap_avg",
    "vwap_std",
    "vwap_median",
    "vwap_qty_sum",
    "hvq_delta",
    "forecast_wind",
    "forecast_solar",
    "forecast_demand",
    "target_two_sided_ptu_realtime",
    "residual_load",
    "dispatch_diff",
    "igcc_diff",
    "is_balanced"
]

features = ['naive_strategy_action', 'min_ptu_price_known',
            'min_price_published', 'forecast_wind',
            'time_since_last_two_sided',
            'two_sided_daily_count', 'mid_price_published',
            'vwap_avg', 'vwap_std', 'vwap_qty_sum',
            'minute_in_ptu', 'downward_dispatch_published',
            'settlement_price_bestguess', 'import_capacity',
            'upward_dispatch_published',
            'settlement_price_realized', 'forecast_solar',
            'igcc_contribution_down_published', 'vwap_max',
# 'min_price_diff',
# 'residual_load',
# 'balance',
# 'import_zero',
# 'dispatch_diff',
# 'downward_dispatch_diff',
# 'import_capacity_left',
# 'igcc_down_diff',
# 'forecast_wind_delta'
]

target = "fix_two_sided_ptu"
version = "target"
train_set = "train"
test_set = "valid"

splits = get_splits(df, features, target, return_dict_pair=False)


In [ ]:
def entropy(labels, base=None):
  """ Computes entropy of label distribution. """

  n_labels = len(labels)

  if n_labels <= 1:
    return 0

  value,counts = np.unique(labels, return_counts=True)
  probs = counts / n_labels
  n_classes = np.count_nonzero(probs)

  if n_classes <= 1:
    return 0

  ent = 0.

  base = math.e if base is None else base
  for i in probs:
    ent -= i * math.log(i, base)

  return ent

def best_split_entropy(df, feature, target):
    """Calculate the information gain for a split on a numerical feature."""
    original_entropy = entropy(df[target])
    best_gain = 0
    best_split = None

    sorted_values = np.sort(df[feature].unique())
    for value in sorted_values:
        left_split = df[df[feature] <= value][target]
        right_split = df[df[feature] > value][target]

        if len(left_split) == 0 or len(right_split) == 0:
            continue

        left_weight = len(left_split) / len(df)
        right_weight = len(right_split) / len(df)
        split_entropy = left_weight * entropy(left_split) + right_weight * entropy(right_split)

        gain = original_entropy - split_entropy
    
        if gain > best_gain:
            best_gain = gain
            best_split = value

    return best_gain, best_split


In [36]:
def hellinger_distance(N_A_parent, N_B_parent, N_A_left, N_B_left, N_A_right, N_B_right):

    term_left = (np.sqrt(N_A_left / N_A_parent) - np.sqrt(N_B_left / N_B_parent)) ** 2
    term_right = (np.sqrt(N_A_right / N_A_parent) - np.sqrt(N_B_right / N_B_parent)) ** 2

    return term_left + term_right
    

def best_split_hellinger(df, feature, target):
    """Find the best split for a numerical feature using the Hellinger distance."""
    best_distance = float('inf')
    best_split = None

    sorted_values = np.sort(df[feature].unique())
    N_A_parent = df[target].sum()
    N_B_parent = len(df) - N_A_parent

    for value in sorted_values:
        left_split = df[df[feature] <= value]
        right_split = df[df[feature] > value]

        if len(left_split) == 0 or len(right_split) == 0:
            continue

        N_A_left = left_split[target].sum()
        N_B_left = len(left_split) - N_A_left
        N_A_right = right_split[target].sum()
        N_B_right = len(right_split) - N_A_right

        distance = hellinger_distance(N_A_parent, N_B_parent, N_A_left, N_B_left, N_A_right, N_B_right)

        if distance < best_distance:
            best_distance = distance
            split_value = value

    return best_distance, split_value

In [100]:
from collections import namedtuple

Node = namedtuple('Node', ['column_name',  'mode', 'branches'])

def create_split(df, column, mode):
    if df[column].unique() == 2:
        split = {'left': df[df[column] == mode], 'right': df[df[column] != mode]}
    else:
        split = {'left': df[df[column] <= mode], 'right': df[df[column] > mode]}

    return split

In [98]:
def ID3(data, target, max_depth=5):

    # step 1
    if data[target].nunique() == 1 or max_depth <= 0:
      print(data[target])
      return data[target].iloc[0]
    print(f"Layers to go = {max_depth}")
    max_depth -= 1

    # step 2
    # check if there's just the target left
    if data.columns[0] == target and len(data.columns) == 1:
      return data[target].mode()

    # step 3
    print(f"step 3")
    distances = []
    split_values = []
    column_list = data.columns.drop(target)
    for column in column_list:
      print(column)
      best_distance, split_value = best_split_hellinger(data, column, target)
      distances.append(best_distance)
      split_values.append(split_value)

    # step 4
    print(f"step 4")
    max_idx = np.argmax(distances)
    mode = split_values[max_idx]

    # step 5
    branches = {}
    tree = Node(column_list[max_idx], mode, branches)

    # step 6
    print(f"step 6")
    the_split = create_split(data, column_list[max_idx], split_values[max_idx])
    for key in the_split.keys():
      df = the_split[key]
      new_branch = ID3(df, target, max_depth)
      branches.update({key: new_branch})

    # step 7
    return tree


In [90]:
test_target = 'target'

test_df = pd.DataFrame([[1, 1, 1, 1],
                        [0, 0, 0, 0],
                        [0, 1, 1, 1]],
                        columns=['f1', 'f2', 'f3', 'target'])

X_test = [[0, 0, 1]]
y_test = [0]

tree = ID3(test_df, test_target)
print(tree)

Layers to go = 5
step 2
step 3
f1
f2
f3
step 4
step 5
step 6
Node(column_name='f2', mode=0, branches={'left': 0, 'right': 1})


In [99]:
df_train = pd.DataFrame(splits[train_set][3][features + [target]])

tree = ID3(df_train[:1000], target, max_depth=2)
print(tree)


Layers to go = 2
step 3
naive_strategy_action
min_ptu_price_known
min_price_published
forecast_wind
time_since_last_two_sided
two_sided_daily_count
mid_price_published
vwap_avg
vwap_std
vwap_qty_sum
minute_in_ptu
downward_dispatch_published
settlement_price_bestguess
import_capacity
upward_dispatch_published
settlement_price_realized
forecast_solar
igcc_contribution_down_published
vwap_max
step 4
step 6
dict_keys(['left', 'right'])
Layers to go = 1
step 3
naive_strategy_action
min_ptu_price_known
min_price_published
forecast_wind
time_since_last_two_sided
two_sided_daily_count
mid_price_published
vwap_avg
vwap_std
vwap_qty_sum
minute_in_ptu
downward_dispatch_published
settlement_price_bestguess
import_capacity
upward_dispatch_published
settlement_price_realized
forecast_solar
igcc_contribution_down_published
vwap_max
step 4
step 6
dict_keys(['left', 'right'])
6       True
7       True
8       True
9       True
10      True
       ...  
899    False
902    False
966     True
973     Tru

IndexError: single positional indexer is out-of-bounds

In [67]:
def classify(tree, row):
    node = tree.column_name
    row_value = row[node]
    branches = tree.branches

    if row_value in branches:

      # another Node is attached
      if isinstance(branches[row_value], Node):
        leaf = classify(branches[row_value], row)
        return leaf

      # this is a leaf node
      elif (isinstance(branches[row_value], np.bool_) or
          isinstance(branches[row_value], bool)):
        return branches[row_value]

      # there is only one key left in branch
      else:
        return branches[list(branches.keys())[0]].iloc[0]

    else:
        return tree.mode